In [ ]:
import getpass
from datetime import datetime, timedelta
from tapipy.tapis import Tapis
from random import gauss

In [ ]:
username = getpass.getpass(prompt = "Username: ", stream=None)
password = getpass.getpass(prompt = "Password: ", stream=None)

base_url = "https://tacc.tapis.io"

client = Tapis(
    base_url = base_url, 
    username = username,
    password = password
) 

#generate access token
client.get_tokens()

In [ ]:
project_id = f"smart_data_workshop_project_{username}"
site_id = f"smart_data_workshop_site_{username}"
inst_id = f"smart_data_workshop_instrument_{username}"

In [ ]:
#define and create project
project = {
    "project_name": project_id,
    "description": f"A smart data workshop project for user {username}",
    "owner": username,
    "pi": username,
    "active": True,
    "metadata": {}
}
client.streams.create_project(**project)

In [ ]:
#define and create site
site = {
    "project_id": project_id,
    "request_body": [{
        "site_name": site_id,
        "site_id": site_id,
        "description": f"A smart data workshop site for user {username}",
        "latitude": 19.89,
        "longitude": 155.58,
        "elevation": 10,
        "metadata": {}
    }]
}
client.streams.create_site(**site)

In [ ]:
#define and create instrument
instrument = {
    "project_id": project_id,
    "site_id": site_id,
    "request_body": [{
        "inst_name": inst_id,
        "inst_id": inst_id,
        "inst_description": f"A smart data workshop instrument for user {username}",
        "metadata": {}
    }]
}
client.streams.create_instrument(**instrument)

In [ ]:
#define and create variables
variables = {
    "project_id": project_id,
    "site_id": site_id,
    "inst_id": inst_id,
    "request_body": [{
        "var_name": "rainfall",
        "var_id": "rainfall",
        "units": "mm",
        "shortname": "rf",
        "metadata": {}
    },
    {
        "var_name": "temperature",
        "var_id": "temperature",
        "units": "C",
        "shortname": "temp",
        "metadata": {}
    }]
}
client.streams.create_variable(**variables)

In [ ]:
#define variable generation information
avg_temp_c = 27
avg_rainfall_mm = 10
rainfall_stddev = 3
temp_stddev = 3

In [ ]:
#generate random measurement data and create measurements
data = []
timestamp = datetime.fromisoformat("2022-01-01T00:00:00")
for i in range(365):
    temp = round(gauss(avg_temp_c, temp_stddev), 2)
    rainfall = max(round(gauss(avg_rainfall_mm, rainfall_stddev), 2), 0)
    timestamp_s = timestamp.isoformat()
    data.append({
        "datetime": timestamp_s,
        "temperature": temp,
        "rainfall": rainfall
    })
    timestamp = timestamp + timedelta(days = 1)
measurements = {
    "inst_id": inst_id,
    "vars": data
}
client.streams.create_measurement(**measurements)

In [ ]:
#define threshold values (values exceeding 2 standard deviations)
temp_thresholds = [avg_temp_c - temp_stddev * 2, avg_temp_c + temp_stddev * 2]
rainfall_thresholds = [avg_rainfall_mm - rainfall_stddev * 2, avg_rainfall_mm + rainfall_stddev * 2]

#get created measurements
measurements_data = client.streams.download_measurements(inst_id = inst_id)

#get data surpassing thresholds
threshold_data = {
    "rainfall": {},
    "temperature": {}
}
for prop, val in vars(measurements_data.rainfall).items():
    if val < rainfall_thresholds[0] or val > rainfall_thresholds[1]:
        threshold_data["rainfall"][prop] = val
    
for prop, val in vars(measurements_data.temperature).items():
    if val < temp_thresholds[0] or val > temp_thresholds[1]:
        threshold_data["temperature"][prop] = val
        
print(threshold_data)

In [ ]:
actor = {
    "image": "mcleanj/outlier_processor",
    "name": "outlier_processor",
    "description": "A container for processing outlier values for measurements",
    "default_environment": {
        "USERNAME": username
    }
}

actor_info = client.actors.createActor(**actor)
actor_id = actor_info.id

In [ ]:
exec_info = client.actors.sendMessage(actor_id = actor_id, request_body = {
    "message": "test message"
})
exec_id = exec_info.executionId

In [ ]:
execution = client.actors.getExecution(actor_id = actor_id, execution_id = exec_id)
print(execution)

In [ ]:
logs = client.actors.getExecutionLogs(actor_id = actor_id, execution_id = exec_id)
print(logs)